### Implemented by Mohammad Awad on 11-Mar-2023

#### Implementation for the PC production planning problem faced in operational research on python CPLEX.

In [338]:
import cplex
import docplex

In [339]:
from docplex.mp.model import Model
m = Model(name = 'PC_Production_Planning')

In [340]:
#sets
T=["R", "OT"] #coressponding to regular and overtime PC production types
W=[1, 2, 3, 4, 5, 6] #coressponding to production weeks

In [341]:
#data
#{"Regular conditions":[capacity, cost], "Overtime conditions":[capacity, cost]}
Cap_Cost_t={"R": [160, 190] , "OT": [50, 260]}
Ow={1:105, 2:170, 3:230, 4:180, 5:150, 6:250} #order demand per week {week:demand}
I=10 #inventory cost per PC held per week

In [342]:
#Decion Variable
x=((w,t) for w in W for t in T)
Xwt=m.integer_var_dict(x, 0, None, name= "Number of PCs for week %s to produce")
Xwt

{(1, 'R'): docplex.mp.Var(type=I,name='Number of PCs for week 1_R to produce'),
 (1,
  'OT'): docplex.mp.Var(type=I,name='Number of PCs for week 1_OT to produce'),
 (2, 'R'): docplex.mp.Var(type=I,name='Number of PCs for week 2_R to produce'),
 (2,
  'OT'): docplex.mp.Var(type=I,name='Number of PCs for week 2_OT to produce'),
 (3, 'R'): docplex.mp.Var(type=I,name='Number of PCs for week 3_R to produce'),
 (3,
  'OT'): docplex.mp.Var(type=I,name='Number of PCs for week 3_OT to produce'),
 (4, 'R'): docplex.mp.Var(type=I,name='Number of PCs for week 4_R to produce'),
 (4,
  'OT'): docplex.mp.Var(type=I,name='Number of PCs for week 4_OT to produce'),
 (5, 'R'): docplex.mp.Var(type=I,name='Number of PCs for week 5_R to produce'),
 (5,
  'OT'): docplex.mp.Var(type=I,name='Number of PCs for week 5_OT to produce'),
 (6, 'R'): docplex.mp.Var(type=I,name='Number of PCs for week 6_R to produce'),
 (6,
  'OT'): docplex.mp.Var(type=I,name='Number of PCs for week 6_OT to produce')}

In [343]:
#Auxiliary variable
Vw=m.integer_var_dict(W, 0, None, "Number of PCs in inventory after week %s")
Vw

{1: docplex.mp.Var(type=I,name='Number of PCs in inventory after week 1'),
 2: docplex.mp.Var(type=I,name='Number of PCs in inventory after week 2'),
 3: docplex.mp.Var(type=I,name='Number of PCs in inventory after week 3'),
 4: docplex.mp.Var(type=I,name='Number of PCs in inventory after week 4'),
 5: docplex.mp.Var(type=I,name='Number of PCs in inventory after week 5'),
 6: docplex.mp.Var(type=I,name='Number of PCs in inventory after week 6')}

In [344]:
#objective function
m.minimize(m.sum(m.sum(Xwt[(w, t)]*Cap_Cost_t[t][1] for t in T) for w in W) + I*m.sum(Vw[w] for w in W))

In [345]:
#Constraints
m.add_constraint(Vw[6]==0)

docplex.mp.LinearConstraint[](Number of PCs in inventory after week 6,EQ,0)

In [346]:
#formulated for each weeks capacity by type
m.add_constraints([Xwt[(w, t)]<=Cap_Cost_t[t][0] for t in T for w in W]) 

[docplex.mp.LinearConstraint[](Number of PCs for week 1_R to produce,LE,160),
 docplex.mp.LinearConstraint[](Number of PCs for week 2_R to produce,LE,160),
 docplex.mp.LinearConstraint[](Number of PCs for week 3_R to produce,LE,160),
 docplex.mp.LinearConstraint[](Number of PCs for week 4_R to produce,LE,160),
 docplex.mp.LinearConstraint[](Number of PCs for week 5_R to produce,LE,160),
 docplex.mp.LinearConstraint[](Number of PCs for week 6_R to produce,LE,160),
 docplex.mp.LinearConstraint[](Number of PCs for week 1_OT to produce,LE,50),
 docplex.mp.LinearConstraint[](Number of PCs for week 2_OT to produce,LE,50),
 docplex.mp.LinearConstraint[](Number of PCs for week 3_OT to produce,LE,50),
 docplex.mp.LinearConstraint[](Number of PCs for week 4_OT to produce,LE,50),
 docplex.mp.LinearConstraint[](Number of PCs for week 5_OT to produce,LE,50),
 docplex.mp.LinearConstraint[](Number of PCs for week 6_OT to produce,LE,50)]

In [347]:
m.add_constraints([m.sum(Xwt[(w,t)] for t in T)+ Vw[w-1] >= Ow[w] for w in W[1:]])

[docplex.mp.LinearConstraint[](Number of PCs for week 2_R to produce+Number of PCs for week 2_OT to produce+Number of PCs in inventory after week 1,GE,170),
 docplex.mp.LinearConstraint[](Number of PCs for week 3_R to produce+Number of PCs for week 3_OT to produce+Number of PCs in inventory after week 2,GE,230),
 docplex.mp.LinearConstraint[](Number of PCs for week 4_R to produce+Number of PCs for week 4_OT to produce+Number of PCs in inventory after week 3,GE,180),
 docplex.mp.LinearConstraint[](Number of PCs for week 5_R to produce+Number of PCs for week 5_OT to produce+Number of PCs in inventory after week 4,GE,150),
 docplex.mp.LinearConstraint[](Number of PCs for week 6_R to produce+Number of PCs for week 6_OT to produce+Number of PCs in inventory after week 5,GE,250)]

In [348]:
m.add_constraint(m.sum(Xwt[(1,t)] for t in T)>= Ow[1])

docplex.mp.LinearConstraint[](Number of PCs for week 1_R to produce+Number of PCs for week 1_OT to produce,GE,105)

In [349]:
m.add_constraints([m.sum(Xwt[(w,t)] for t in T) + Vw[w-1] - Ow[w] == Vw[w] for w in W[1:]])

[docplex.mp.LinearConstraint[](Number of PCs for week 2_R to produce+Number of PCs for week 2_OT to produce+Number of PCs in inventory after week 1-170,EQ,Number of PCs in inventory after week 2),
 docplex.mp.LinearConstraint[](Number of PCs for week 3_R to produce+Number of PCs for week 3_OT to produce+Number of PCs in inventory after week 2-230,EQ,Number of PCs in inventory after week 3),
 docplex.mp.LinearConstraint[](Number of PCs for week 4_R to produce+Number of PCs for week 4_OT to produce+Number of PCs in inventory after week 3-180,EQ,Number of PCs in inventory after week 4),
 docplex.mp.LinearConstraint[](Number of PCs for week 5_R to produce+Number of PCs for week 5_OT to produce+Number of PCs in inventory after week 4-150,EQ,Number of PCs in inventory after week 5),
 docplex.mp.LinearConstraint[](Number of PCs for week 6_R to produce+Number of PCs for week 6_OT to produce+Number of PCs in inventory after week 5-250,EQ,Number of PCs in inventory after week 6)]

In [350]:
m.add_constraint(m.sum(Xwt[(1,t)] for t in T) - Ow[1] ==Vw[1])

docplex.mp.LinearConstraint[](Number of PCs for week 1_R to produce+Number of PCs for week 1_OT to produce-105,EQ,Number of PCs in inventory after week 1)

In [351]:
m.export_as_lp('PC_Production_Planning.lp')

'PC_Production_Planning.lp'

In [352]:
m.print_information()

Model: PC_Production_Planning
 - number of variables: 18
   - binary=0, integer=18, continuous=0
 - number of constraints: 25
   - linear=25
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [353]:
m.solve()
print(m.solve_status)
m.print_solution()


JobSolveStatus.OPTIMAL_SOLUTION
objective: 216300
  "Number of PCs for week 1_R to produce"=160
  "Number of PCs for week 2_R to produce"=160
  "Number of PCs for week 3_R to produce"=160
  "Number of PCs for week 3_OT to produce"=25
  "Number of PCs for week 4_R to produce"=160
  "Number of PCs for week 4_OT to produce"=20
  "Number of PCs for week 5_R to produce"=160
  "Number of PCs for week 5_OT to produce"=30
  "Number of PCs for week 6_R to produce"=160
  "Number of PCs for week 6_OT to produce"=50
  "Number of PCs in inventory after week 1"=55
  "Number of PCs in inventory after week 2"=45
  "Number of PCs in inventory after week 5"=40
